<a href="https://colab.research.google.com/github/ancestor9/Gyeongbok-AI-study/blob/main/2025%20AI%20Tutor/pdf_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 이미지형태의 pdf파일을 캡쳐하여 PNG 파일로 만든 후

In [ ]:
!pip install langchain-community --quiet

In [19]:
import openai
import base64
from PIL import Image
from io import BytesIO
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage
from google.colab import userdata

# API key setup
openai_api_key = userdata.get('new_open_ai')
openai.api_key = openai_api_key

# Function to convert image to base64
def image_to_base64(img_path):
    with Image.open(img_path) as img:
        buffered = BytesIO()
        img.save(buffered, format="PNG")
        return base64.b64encode(buffered.getvalue()).decode()

# Image file path
img_path = "/content/sample_file.png"
encoded_img = image_to_base64(img_path)

# GPT-4o model with Vision support
vision_model = ChatOpenAI(
    model="gpt-4o",
    openai_api_key=openai.api_key,
    max_tokens=2048
)

# Configure the message with clear instructions
messages = [
    SystemMessage(content="Please extract and transcribe ALL text content from the image exactly as it appears, including ALL question text, ALL choices, and ALL bullet points. DO NOT analyze, interpret, or select answers. Just output the complete text content."),
    HumanMessage(content=[
        {
            "type": "image_url",
            "image_url": {
                "url": f"data:image/png;base64,{encoded_img}"
            }
        }
    ])
]

# Call for response
response = vision_model.invoke(messages)

# Output the result
print(response.content)

1. 다음에서 설명하는 인체 기본조직은?
- 몸에 널리 분포하며, 영양 구조로 이룸
- 세포나 기관 사이 틈을 메우고, 기관들 지지·보호함

① 근육조직  
② 신경조직  
③ 상피조직  
④ 결합조직  
⑤ 혈액조직  

2. 다음에서 설명하는 뼈는?
- 앞면에 가로선(transverse line)이 있음
- 위쪽에 넓고 아래쪽으로 좁은 사다리꼴 모양
- 가쪽에 귀모양의 귀면(auricular surface)이 있음

① 망치뼈  
② 엉치뼈  
③ 위팔뼈  
④ 무릎뼈  
⑤ 넙다리뼈  

3. 다음에서 설명하는 머리뼈는?
- 체판, 수직판, 미로 등으로 구성
- 체판구멍을 통해 후각신경이 통과

① 코뼈  
② 나비뼈  
③ 벌집뼈  
④ 이마뼈  
⑤ 뒤통수뼈  

4. 다음에서 설명하는 관절은?
- 굽힘, 폄, 모음, 벌림 가능
- 손목관절, 손허리손가락관절, 고리돌기사이관절에 해당

① 경첩관절  
② 평면관절  
③ 타원관절  
④ 공모양관절  
⑤ 안장관절  


## 추출한 텍스트를 파일로 저장

In [ ]:
with open("output.txt", "w") as f:
  f.write(response.content)

print("Response content saved to output.txt")

## 텍스트 파일을 Document 형식으로 체계적으로 변환
- 각 문제를 개별 Document로 구조화하여 저장하는 방식

In [21]:
from langchain.schema import Document
import re

def parse_questions_from_file(file_path):
    """텍스트 파일에서 문제를 읽고 Document 객체 리스트로 변환"""

    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # 문제 단위로 분리
    # 숫자와 점으로 시작하는 패턴을 찾아 문제 분리
    question_pattern = r'(\d+)\.\s+(.*?)(?=\d+\.\s+|$)'
    question_matches = re.finditer(question_pattern, content, re.DOTALL)

    documents = []

    for match in question_matches:
        question_number = int(match.group(1))
        question_content = match.group(2).strip()

        # 질문과 설명, 보기 분리
        parts = question_content.split('\n\n')

        # 질문 추출 (첫 번째 줄)
        question_text = parts[0].split('\n')[0].strip()

        # 설명 추출 (- 로 시작하는 줄)
        description_lines = []
        description_pattern = r'- (.*)'
        for line in parts[0].split('\n')[1:]:
            desc_match = re.match(description_pattern, line.strip())
            if desc_match:
                description_lines.append(desc_match.group(1).strip())

        # 보기 추출
        options = {}
        option_pattern = r'([①-⑮])\s+(.*)'

        # 보기를 찾을 부분 (첫 번째 파트 이후)
        options_text = parts[1] if len(parts) > 1 else ""

        for line in options_text.split('\n'):
            line = line.strip()
            if not line:
                continue

            option_match = re.match(option_pattern, line)
            if option_match:
                # 한글 숫자를 아라비아 숫자로 변환
                korean_number = option_match.group(1)
                # 한글 숫자 매핑
                korean_to_arabic = {
                    '①': 1, '②': 2, '③': 3, '④': 4, '⑤': 5,
                    '⑥': 6, '⑦': 7, '⑧': 8, '⑨': 9, '⑩': 10,
                    '⑪': 11, '⑫': 12, '⑬': 13, '⑭': 14, '⑮': 15
                }
                arabic_number = korean_to_arabic.get(korean_number, 0)
                options[arabic_number] = option_match.group(2).strip()

        # Document 객체 생성을 위한 메타데이터 구성
        metadata = {
            "number": question_number,
            "question": question_text,
            "description": description_lines,
            "options": options
        }

        # Document 객체 생성 및 리스트에 추가
        documents.append(Document(
            page_content=question_content,
            metadata=metadata
        ))

    return documents

def save_documents_to_json(documents, output_file):
    """Document 객체 리스트를 JSON 파일로 저장"""
    import json

    # Document 객체를 직렬화 가능한 딕셔너리로 변환
    serializable_docs = []
    for doc in documents:
        serializable_docs.append({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        })

    # JSON 파일로 저장
    with open(output_file, 'w', encoding='utf-8') as f:
        json.dump(serializable_docs, f, ensure_ascii=False, indent=2)

    return output_file

# 전체 코드 실행 예시
def process_questions_file(input_file_path, output_json_path):
    """텍스트 파일에서 문제를 추출하고 JSON으로 저장"""
    # 문제 추출 및 Document 객체 생성
    documents = parse_questions_from_file(input_file_path)

    # 결과 출력 (확인용)
    print(f"총 {len(documents)}개의 문제를 추출했습니다.")
    for i, doc in enumerate(documents, 1):
        print(f"\n문제 {i}:")
        print(f"- 번호: {doc.metadata['number']}")
        print(f"- 질문: {doc.metadata['question']}")
        print("- 설명:")
        for desc in doc.metadata['description']:
            print(f"  • {desc}")
        print("- 보기:")
        for num, option in sorted(doc.metadata['options'].items()):
            print(f"  {num}. {option}")

    # JSON 파일로 저장
    saved_file = save_documents_to_json(documents, output_json_path)
    print(f"\n문제가 '{saved_file}' 파일에 저장되었습니다.")

    return documents

# 실행 예시
input_file = "output.txt"  # 텍스트 파일 경로
output_file = "questions.json"  # 출력할 JSON 파일 경로
questions = process_questions_file(input_file, output_file)

총 4개의 문제를 추출했습니다.

문제 1:
- 번호: 1
- 질문: 다음에서 설명하는 인체 기본조직은?
- 설명:
  • 몸에 널리 분포하며, 영양 구조로 이룸
  • 세포나 기관 사이 틈을 메우고, 기관들 지지·보호함
- 보기:
  1. 근육조직
  2. 신경조직
  3. 상피조직
  4. 결합조직
  5. 혈액조직

문제 2:
- 번호: 2
- 질문: 다음에서 설명하는 뼈는?
- 설명:
  • 앞면에 가로선(transverse line)이 있음
  • 위쪽에 넓고 아래쪽으로 좁은 사다리꼴 모양
  • 가쪽에 귀모양의 귀면(auricular surface)이 있음
- 보기:
  1. 망치뼈
  2. 엉치뼈
  3. 위팔뼈
  4. 무릎뼈
  5. 넙다리뼈

문제 3:
- 번호: 3
- 질문: 다음에서 설명하는 머리뼈는?
- 설명:
  • 체판, 수직판, 미로 등으로 구성
  • 체판구멍을 통해 후각신경이 통과
- 보기:
  1. 코뼈
  2. 나비뼈
  3. 벌집뼈
  4. 이마뼈
  5. 뒤통수뼈

문제 4:
- 번호: 4
- 질문: 다음에서 설명하는 관절은?
- 설명:
  • 굽힘, 폄, 모음, 벌림 가능
  • 손목관절, 손허리손가락관절, 고리돌기사이관절에 해당
- 보기:
  1. 경첩관절
  2. 평면관절
  3. 타원관절
  4. 공모양관절
  5. 안장관절

문제가 'questions.json' 파일에 저장되었습니다.
